### ***Problem05***
---

In [1]:
### Import library & data set
# library
from google.colab import drive
drive.mount("/content/drive")
import sys
import numpy as np
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# data set
location_spiral = "/content/drive/MyDrive/DS5230_Unsupervised Machine Learning/Assignments/HW03A/twospirals.txt"
location_circle = "/content/drive/MyDrive/DS5230_Unsupervised Machine Learning/Assignments/HW03A/threecircles.txt"
df_spiral = pd.read_csv(location_spiral, sep="\t", header=None)
df_circle = pd.read_csv(location_circle, sep=",", header=None)
def to_df(data):
  df = pd.DataFrame(data)
  df.rename(columns={0:"x1",1:"x2",2:"y"}, inplace=True)
  return df
# df_circle = pd.read_csv(location_circle, sep=",", header=None)
df_spiral = to_df(df_spiral)
df_circle = to_df(df_circle)

Mounted at /content/drive


In [ ]:
### kernel
def gaussian_kernel(X, Y, sigma):
    pairwise_sq_dists = np.sum(X**2, axis=1, keepdims=True) + np.sum(Y**2, axis=1) - 2 * np.dot(X, Y.T)
    return np.exp(-pairwise_sq_dists / (2 * sigma**2))

def kernel_pca(X, num_components, sigma=1.0):
    # kernel
    K = gaussian_kernel(X, X, sigma)
    # centralize
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    K_centered = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    # eigen vector, value
    eigenvalues, eigenvectors = np.linalg.eigh(K_centered)
    # sorting
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    # select
    components = eigenvectors[:, :num_components]
    # result
    projected_data = np.dot(K_centered, components)
    return projected_data

In [ ]:
### (A)
# spiral - linear regression
X_train, X_test, y_train, y_test = train_test_split(df_spiral[["x1", "x2"]], df_spiral["y"], test_size=0.3, random_state=22)
reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
y_predict = reg.predict(X_test)
rmse = round(mean_squared_error(y_test, y_predict), 2)
print("sprial data rmse:", rmse)
# three circles - linear regression
X_train, X_test, y_train, y_test = train_test_split(df_circle[["x1", "x2"]], df_circle["y"], test_size=0.3, random_state=22)
reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
y_predict = reg.predict(X_test)
rmse = round(mean_squared_error(y_test, y_predict), 2)
print("circle data rmse:", rmse)

sprial data rmse: 0.86
circle data rmse: 0.67


In [ ]:
### (B), (C)
# spiral - kernel pca
X_train, X_test, y_train, y_test = train_test_split(df_spiral[["x1", "x2"]], df_spiral["y"], test_size=0.3, random_state=22)
X_train = kernel_pca(np.array(X_train), num_components=20)
reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
y_predict = reg.predict(X_train)
rmse = round(mean_squared_error(y_train, y_predict), 2)
print("Required d: 20","sprial data rmse:", rmse)
# circle - kernel pca
X_train, X_test, y_train, y_test = train_test_split(df_circle[["x1", "x2"]], df_circle["y"], test_size=0.3, random_state=22)
X_train = kernel_pca(np.array(X_train), num_components=10)
reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
y_predict = reg.predict(X_train)
rmse = round(mean_squared_error(y_train, y_predict), 2)
print("Required d: 10","circle data rmse:", rmse)

Required d: 20 sprial data rmse: 0.24
Required d: 10 circle data rmse: 0.02
